In [1]:
#Libraries
import pypyodbc
import pandas as pd
#Conn and credentials
conn_str = 'Driver={ODBC Driver 18 for SQL Server};Server=tcp:servidortcprueba.database.windows.net,1433;Database=db_campechanos;Uid=adminraul;Pwd=5ccu9$j$XzhNMMF;Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;'
connection = pypyodbc.connect(conn_str)
cursor = connection.cursor()

#Training dataset
query = "SELECT * FROM dev_train"
cursor.execute(query)
rows = cursor.fetchall()
df_train = pd.DataFrame(rows)

columns = [column[0] for column in cursor.description]
df_train.columns = columns

#Test dataset, dont have sales price
query = "SELECT * FROM dev_test"
cursor.execute(query)
rows = cursor.fetchall()
df_test = pd.DataFrame(rows)

columns = [column[0] for column in cursor.description]
df_test.columns = columns

#Sample dataset, id and sales price
query = "SELECT * FROM sample"
cursor.execute(query)
rows = cursor.fetchall()
df_sample = pd.DataFrame(rows)

columns = [column[0] for column in cursor.description]
df_sample.columns = columns

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import itertools

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pylab import rcParams
import matplotlib

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# TensorFlow 2.x does not require setting verbosity
# tf.logging.set_verbosity(tf.logging.INFO)

# TensorFlow 2.x uses eager execution by default, so you don't need to create an InteractiveSession
# sess = tf.InteractiveSession()

train = df_train
print('Shape of the train data with all features:', train.shape)
train = train.select_dtypes(exclude=['object'])
print("")
print('Shape of the train data with numerical features:', train.shape)

train=train.replace('NA',0)

test_list = train.columns.tolist()
test_list.remove('saleprice')
test = df_test[test_list]
ID = df_test.id
test= test.replace('NA',0)


print("")
print("List of features contained in our dataset:", list(train.columns))


Shape of the train data with all features: (1460, 81)

Shape of the train data with numerical features: (1460, 32)

List of features contained in our dataset: ['lotarea', 'overallqual', 'overallcond', 'yearbuilt', 'yearremodadd', 'bsmtfinsf1', 'bsmtfinsf2', 'bsmtunfsf', 'totalbsmtsf', '_1stflrsf', '_2ndflrsf', 'lowqualfinsf', 'grlivarea', 'bsmtfullbath', 'fullbath', 'halfbath', 'bedroomabvgr', 'kitchenabvgr', 'totrmsabvgrd', 'fireplaces', 'garagecars', 'garagearea', 'wooddecksf', 'openporchsf', 'enclosedporch', '_3ssnporch', 'screenporch', 'poolarea', 'miscval', 'mosold', 'yrsold', 'saleprice']


In [3]:
from sklearn.ensemble import IsolationForest

clf = IsolationForest(max_samples = 100, random_state = 42)
clf.fit(train)
y_noano = clf.predict(train)
y_noano = pd.DataFrame(y_noano, columns = ['Top'])
y_noano[y_noano['Top'] == 1].index.values

train = train.iloc[y_noano[y_noano['Top'] == 1].index.values]
train.reset_index(drop = True, inplace = True)
print("Number of Outliers:", y_noano[y_noano['Top'] == -1].shape[0])
print("Number of rows without outliers:", train.shape[0])

Number of Outliers: 159
Number of rows without outliers: 1301


In [4]:
import warnings
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

warnings.filterwarnings('ignore')

col_train = list(train.columns)
col_train_bis = list(train.columns)

col_train_bis.remove('saleprice')

mat_train = np.asarray(train)  # Convert to numpy.array
mat_test = np.asarray(test)  # Convert to numpy.array
mat_new = np.asarray(train.drop('saleprice', axis=1))  # Convert to numpy.array

# Ensure that the number of rows in mat_y matches the number of rows in train
mat_y = np.asarray(train['saleprice']).reshape((-1, 1))  # Convert to numpy.array

prepro_y = MinMaxScaler()
prepro_y.fit(mat_y)

prepro = MinMaxScaler()
prepro.fit(mat_train)

prepro_test = MinMaxScaler()
prepro_test.fit(mat_new)

train = pd.DataFrame(prepro.transform(mat_train), columns=col_train)
test = pd.DataFrame(prepro_test.transform(mat_test), columns=col_train_bis)

train.head()



,lotarea,overallqual,overallcond,yearbuilt,yearremodadd,bsmtfinsf1,bsmtfinsf2,bsmtunfsf,totalbsmtsf,_1stflrsf,...,wooddecksf,openporchsf,enclosedporch,_3ssnporch,screenporch,poolarea,miscval,mosold,yrsold,saleprice
0,0.045339,0.666667,0.500,0.956204,0.883333,0.375532,0.0,0.073457,0.339144,0.200384,...,0.000000,0.167582,0.000000,0.0,0.0,0.0,0.0,0.090909,0.50,0.427956
1,0.052632,0.555556,0.875,0.759124,0.433333,0.520213,0.0,0.139079,0.500000,0.395014,...,0.404891,0.000000,0.000000,0.0,0.0,0.0,0.0,0.363636,0.25,0.361238
2,0.063094,0.666667,0.500,0.941606,0.866667,0.258511,0.0,0.212537,0.364501,0.231064,...,0.000000,0.115385,0.000000,0.0,0.0,0.0,0.0,0.727273,0.50,0.465021
3,0.052315,0.666667,0.500,0.313869,0.333333,0.114894,0.0,0.264447,0.299525,0.250719,...,0.000000,0.096154,0.824242,0.0,0.0,0.0,0.0,0.090909,0.00,0.258690
4,0.082181,0.777778,0.500,0.934307,0.833333,0.348404,0.0,0.239961,0.453645,0.338926,...,0.260870,0.230769,0.000000,0.0,0.0,0.0,0.0,1.000000,0.50,0.530504


In [5]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

# List of features
COLUMNS = col_train
FEATURES = col_train_bis
LABEL = "saleprice"

# Columns for TensorFlow 2.x
feature_cols = [tf.feature_column.numeric_column(k) for k in FEATURES]

# Training set and Prediction set with the features to predict
training_set = train[COLUMNS]
prediction_set = train.saleprice

# Train and Test split
x_train, x_test, y_train, y_test = train_test_split(training_set[FEATURES], prediction_set, test_size=0.33, random_state=42)
y_train = pd.DataFrame(y_train, columns=[LABEL])

# Create DataFrames for training data
training_set = pd.DataFrame(x_train, columns=FEATURES)
training_set[LABEL] = y_train

# Display the head of the training set
print(training_set.head())


       lotarea  overallqual  overallcond  yearbuilt  yearremodadd  bsmtfinsf1  \
834   0.047685     0.444444        0.625   0.131387      0.766667    0.578723   
1073  0.029803     0.444444        0.875   0.386861      0.783333    0.000000   
5     0.055701     0.777778        0.500   0.963504      0.916667    0.728191   
100   0.012238     0.555556        0.500   0.927007      0.816667    0.222872   
424   0.074984     0.555556        0.500   0.912409      0.800000    0.528723   

      bsmtfinsf2  bsmtunfsf  totalbsmtsf  _1stflrsf  ...  wooddecksf  \
834          0.0   0.000000     0.431062   0.359540  ...    0.065217   
1073         0.0   0.432909     0.350238   0.213806  ...    0.000000   
5            0.0   0.155240     0.667987   0.602109  ...    0.346467   
100          0.0   0.151812     0.288827   0.139501  ...    0.000000   
424          0.0   0.237023     0.585578   0.505753  ...    0.190217   

      openporchsf  enclosedporch  _3ssnporch  screenporch  poolarea  miscval  \


In [6]:
# Same thing but for the test set
y_test = pd.DataFrame(y_test, columns = [LABEL])
testing_set = pd.DataFrame(x_test, columns = FEATURES).merge(y_test, left_index = True, right_index = True)
testing_set.head()

,lotarea,overallqual,overallcond,yearbuilt,yearremodadd,bsmtfinsf1,bsmtfinsf2,bsmtunfsf,totalbsmtsf,_1stflrsf,...,wooddecksf,openporchsf,enclosedporch,_3ssnporch,screenporch,poolarea,miscval,mosold,yrsold,saleprice
478,0.017489,0.666667,0.500,0.919708,0.800000,0.310638,0.123336,0.000000,0.286450,0.157718,...,0.163043,0.109890,0.000000,0.000000,0.0,0.0,0.0,0.454545,0.25,0.241393
722,0.047660,0.555556,0.750,0.722628,0.350000,0.268085,0.000000,0.247796,0.400158,0.294343,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.454545,1.00,0.295756
312,0.063088,0.555556,0.500,0.897810,0.750000,0.177660,0.482697,0.157689,0.475436,0.365292,...,0.000000,0.071429,0.000000,0.000000,0.0,0.0,0.0,0.636364,0.25,0.351354
661,0.025967,0.777778,0.500,0.883212,0.716667,0.389894,0.000000,0.295788,0.529715,0.430968,...,0.130435,0.000000,0.212121,0.330709,0.0,0.0,0.0,0.818182,0.50,0.357531
969,0.045935,0.222222,0.375,0.569343,0.000000,0.000000,0.000000,0.000000,0.000000,0.288591,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.454545,0.75,0.142304


In [7]:
#y_predict = regressor.predict(input_fn=lambda: input_fn(test, pred = True))

def to_submit(pred_y,name_out):
    y_predict = list(itertools.islice(pred_y, test.shape[0]))
    y_predict = [item['predictions'][0] for item in y_predict]
    y_predict = pd.DataFrame(prepro_y.inverse_transform(np.array(y_predict).reshape(len(y_predict),1)), columns = ['SalePrice'])
    y_predict = y_predict.join(ID)
    y_predict.to_csv(name_out + '.csv',index=False)

#to_submit(y_predict, "submission_continuous")

In [8]:

def input_fn(data_set, pred = False):

    if pred == False:

        feature_cols = {k: tf.constant(data_set[k].values) for k in FEATURES}
        labels = tf.constant(data_set[LABEL].values)

        return feature_cols, labels

    if pred == True:
        feature_cols = {k: tf.constant(data_set[k].values) for k in FEATURES}

        return feature_cols

In [54]:
# Model
regressor = tf.estimator.DNNRegressor(feature_columns=feature_cols,
                                          activation_fn = tf.nn.elu, hidden_units=[300, 150, 75, 35, 15,8])

# Deep Neural Network Regressor with the training set which contain the data split by train test split
regressor.train(input_fn=lambda: input_fn(training_set), steps=65000)

# Evaluation on the test set created by train_test_split
ev = regressor.evaluate(input_fn=lambda: input_fn(testing_set), steps=1)

loss_score3 = ev["loss"]

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\rauli\\AppData\\Local\\Temp\\tmpoihctsvo', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorf

In [55]:
print("Final Loss on the testing set with Elu: {0:f}".format(loss_score3))

Final Loss on the testing set with Elu: 0.002540


In [56]:
# Predictions
y_predict = regressor.predict(input_fn=lambda: input_fn(test, pred = True))
to_submit(y_predict, "Elu")

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\rauli\AppData\Local\Temp\tmpoihctsvo\model.ckpt-65000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [34]:

serving_input_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(
  tf.feature_column.make_parse_example_spec(feature_cols))
modelBasePath = "./"

# Guardar el modelo
regressor.export_saved_model(modelBasePath, serving_input_fn)


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: ['serving_default', 'regression']
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['predict']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Restoring parameters from C:\Users\rauli\AppData\Local\Temp\tmpahnjpzav\model.ckpt-65000
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: .\temp-1694714479\saved_model.pb


b'./1694714479'

In [48]:
class PredictableModel(tf.keras.Model):
    def _init_(self, original_model, **kwargs):
        super(PredictableModel, self).__init__(**kwargs)
        self.original_model = original_model

    def predict(self, x):
        # Asumimos que el original_model tiene una función `call` que permite inferencia
        return self.original_model(x)
from sklearn.metrics import mean_squared_error
graph = tf.Graph()
with graph.as_default():
    x = tf.constant(1.0)
    y = tf.constant(2.0)
    
    loaded_regressor = tf.saved_model.load("C:/Users/rauli/OneDrive/Documentos/Python Programs/IA-House-Prices/models/house-prices-advanced-regression-techniques/1694714479")
    wrapped_model = PredictableModel(loaded_regressor)
    predictions = wrapped_model.predict(testing_set)

AttributeError: 'PredictableModel' object has no attribute 'original_model'

In [63]:
a=testing_set.columns.tolist()
a.remove('saleprice')
df_test[a].to_csv('testing_set2_EX.csv')

In [ ]:
# Función para crear la función de entrada (input_fn) para el modelo
def input_fn(data_df, num_epochs, shuffle, batch_size,feature_cols):
    # Convierte las características y las etiquetas a tensores
    feature_cols = {col: tf.convert_to_tensor(data_df[col]) for col in feature_cols}
    labels = tf.convert_to_tensor(data_df['saleprice'])  # Ajusta 'target_column' a tu etiqueta

    # Crea un dataset a partir de las características y etiquetas
    dataset = tf.data.Dataset.from_tensor_slices((feature_cols, labels))

    # Opcional: baraja y repite los datos para entrenamiento
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(data_df))
    dataset = dataset.repeat(num_epochs)
    
    # Opcional: agrupa los datos en lotes
    dataset = dataset.batch(batch_size)

    return dataset

In [ ]:
'''# Modelo
regressor = tf.estimator.DNNRegressor(feature_columns=feature_cols,
                                          activation_fn = tf.nn.elu, hidden_units=[300, 150, 75, 35, 20])

# Entrenamiento
regressor.train(
    input_fn=lambda: input_fn(training_set, num_epochs=100, shuffle=True, batch_size=32),
    steps=50000  # Puedes aumentar este número
)

# Evaluación
ev = regressor.evaluate(
    input_fn=lambda: input_fn(testing_set, num_epochs=1, shuffle=False, batch_size=32),
    steps=1
)

loss_score = ev["loss"]

print(f"Loss Score: {loss_score}")'''